# Robot follower

In [ ]:
# In this part of the project we will present the robot's ability to track and shoot a robot.
# First we want to control the chassis the and gimbal manual, after that we will Enable detection of robot.
# We will set the gimbal rotation speed to the maximum possible, for fast object.
# We will set the travel mode to free mode so we can automatically rotate the gimbal.
# To truck after the robot we will need to save his previous possitions and his current position.
# After that as long as we recognize the robot we will track his location and shoot at him non-stop,
# as soon as the robot comes out of the robot's mediator of vision the robot will stop and wait (to use as little battery of the robot as possible).


pid = rm_ctrl.PIDCtrl()


def start():
    V_avg = 1
    k = 0.65
    pid = rm_ctrl.PIDCtrl()

    # enable detection of S1 robots.
    vision_ctrl.enable_detection(rm_define.vision_detection_car)
    vision_ctrl.set_marker_detection_distance(2)
    # gimbal_ctrl.rotate_with_degree(rm_define.gimbal_up, 20)

    gun_ctrl.set_fire_count(1)

    gimbal_ctrl.set_rotate_speed(540)
    pid.set_ctrl_params(330, 0, 28)

    chassis_ctrl.enable_stick_overlay()
    gimbal_ctrl.enable_stick_overlay()
    robot_ctrl.set_mode(rm_define.robot_mode_free)

    x = 0.0
    y = 0.0

    while True:
        # Get list of detected S1 robots.
        robotList = RmList(vision_ctrl.get_car_detection_info())
        if robotList[1] > 0:
            # We found at least one robot.
            robot_ctrl.set_mode(rm_define.robot_mode_chassis_follow)
            # gimbal.rotate_with_speed(pid.get_output(),0)
            chassis_ctrl.set_trans_speed(0.3)
            chassis_ctrl.move(0)

            # Keep track of the current and previous x and y possitions of the
            # detected robot.
            prevX = x
            prevY = y

            # Get coordinates to the center of the first
            # detected S1 robot. This will be our target.
            x = robotList[2]
            y = robotList[3]

            if abs(x - prevX) < 0.03 and abs(y - prevY) < 0.03:
                continue

            yawAngle = gimbal_ctrl.get_axis_angle(rm_define.gimbal_axis_yaw)
            pitchAngle = gimbal_ctrl.get_axis_angle(rm_define.gimbal_axis_pitch)

            # Compute yaw and pitch angle offsets  how much we need to move
            yaw = 96 * (x - 0.5)
            pitch = 54 * (0.5 - y)

            if abs(yaw) > 2 or abs(pitch) > 2:
                # Point gimbal to target.
                try:
                    gimbal_ctrl.angle_ctrl(yawAngle + yaw, pitchAngle + pitch)
                except:
                    print(" ")
                gun_ctrl.fire_once()
        else:
            # gimbal.rotate_with_speed(0,0)
            chassis_ctrl.stop()

# Shot and person follower

In [ ]:
#################################################################################################################
#                                           RoboMaster Project                                                  #
#                                                                                                               #
#                                   Amichai Kafka & Liav weiss & Omer Michael                                   #
#################################################################################################################

# In this part of the project we will present the robot's ability to track and shoot a person.
# First we want to control the chassis the and gimbal manual, after that we will Enable detection of persons.
# We will set the gimbal rotation speed to the maximum possible, for fast object.
# We will set the travel mode to free mode so we can automatically rotate the gimbal.
# To truck after the persom we will need to save his previous possitions and his current position.
# After that as long as we recognize the human we will track his location and shoot at him non-stop,
# as soon as the person comes out of the robot's mediator of vision the robot will stop and wait (to use as little battery of the robot as possible).

pid = rm_ctrl.PIDCtrl()
pid_Pitch = rm_ctrl.PIDCtrl()
pid_Yaw = rm_ctrl.PIDCtrl()


def start():
    # Enable manual control of chassis and gimbal and free travel mode.
    chassis_ctrl.enable_stick_overlay()
    gimbal_ctrl.enable_stick_overlay()
    robot_ctrl.set_mode(rm_define.robot_mode_free)
    V_avg = 1
    k = 0.65
    gun_ctrl.set_fire_count(1)
    # Enable detection of persons.
    vision_ctrl.enable_detection(rm_define.vision_detection_people)
    vision_ctrl.set_marker_detection_distance(1)

    gimbal_ctrl.set_rotate_speed(540)
    pid.set_ctrl_params(330, 0, 28)
    pid_Yaw.set_ctrl_params(330, 0, 28)
    pid_Pitch.set_ctrl_params(330, 0, 28)

    x = 0.0
    y = 0.0

    while True:
        # Get list of detected persons.
        person_list = RmList(vision_ctrl.get_people_detection_info())
        if person_list[1] > 0:

            # We found at least one person.
            robot_ctrl.set_mode(rm_define.robot_mode_chassis_follow)
            chassis_ctrl.set_trans_speed(0.3)
            chassis_ctrl.move(0)
            # Keep track of the current and previous x and y possitions of the
            # detected person.
            prevX = x
            prevY = y

            # Get coordinates to the center of the first detected person.
            x = person_list[2]
            y = person_list[3]

            #if the targate close to the prev position continue
            if abs(x - prevX) < 0.03 and abs(y - prevY) < 0.03:
                continue

            print('x=', x, 'y=', y)

            # Obtain Compute gimbal angle in relation to the chassis.
            yaw_angle = gimbal_ctrl.get_axis_angle(rm_define.gimbal_axis_yaw)
            pitch_angle = gimbal_ctrl.get_axis_angle(rm_define.gimbal_axis_pitch)

            # Compute yaw and pitch angle offsets how much we need to move.
            yaw = 100 * (x - 0.5)
            pitch = 50 * (0.5 - y)
            print('yaw=', yaw, 'pitch=', pitch)
            # pid_Yaw.set_error(x - 0.5)
            # pid_Pitch.set_error(0.5 - y)
            if abs(yaw) > 2 or abs(pitch) > 2:
                # Point gimbal to target.
                gimbal_ctrl.angle_ctrl(yaw_angle + yaw, pitch_angle + pitch)
                gun_ctrl.fire_once()
        else:
            # gimbal.rotate_with_speed(0,0)
            chassis_ctrl.stop()

# Line follower

In [ ]:
# In this part of the project we will present the ability of the robot to follow a line.
# In this code the robot will follow a red line, but you can easily change the color
# of the line that the robot is following using line_follow_color_<color> located on line 30.
# First we lower the gimbal to look at the floor and after that as soon as the robot detects the red line it will move towards it,
# we have chosen the speed of the robot to be slower so that the robot can make sharp turns.

pid=rm_ctrl.PIDCtrl()

def start():
    robot_ctrl.set_mode(rm_define.robot_mode_gimbal_follow)
    gimbal_ctrl.rotate_with_degree(rm_define.gimbal_down,20)
    vision_ctrl.enable_detection(rm_define.vision_detection_line)
    vision_ctrl.line_follow_color_set(rm_define.line_follow_color_red)
    media_ctrl.exposure_value_update(rm_define.exposure_value_small)
    V_avg=1
    k=0.65

    pid.set_ctrl_params(330,0,28)
    while True:
        LineList=RmList(vision_ctrl.get_line_detection_info())
        print(LineList)
        if len(LineList) == 42:
            if LineList[2] >= 1:
                robot_ctrl.set_mode(rm_define.robot_mode_chassis_follow)
                variable_x = LineList[19]
                pid.set_error(variable_x - 0.5)
                gimbal_ctrl.rotate_with_speed(pid.get_output(),0)
                v=(V_avg-(k*abs(LineList[37]/180)))
                chassis_ctrl.set_trans_speed(v*0.5)
                chassis_ctrl.move(0)
        else:
            gimbal_ctrl.rotate_with_speed(0,0)
            # chassis_ctrl.stop()


# Marker color detection

To use the vision_ctrl.marker_detection_color_set() function in the RoboMaster Python SDK, you need to specify the color of the visual labels that the robot should identify. This function allows you to set the color for marker detection, which is useful when you have markers of different colors and you want the robot to focus on a specific one.

- Available Colors
The color_enum parameter specifies the color for marker detection. The available colors are typically defined in the SDK as constants within rm_define. Here are some common color enums you might find:
- rm_define.marker_detection_color_red
- rm_define.marker_detection_color_blue
- rm_define.marker_detection_color_green
- rm_define.marker_detection_color_yellow


﻿=================== Vision ===================

.. function:: vision_ctrl.marker_detection_color_set(color_enum)

    :description: Sets the color for identifying visual labels

    :param color_enum: For available label colors, refer to the :data:`color_enum` table.

    :return: None

    :example: ``vision_ctrl.marker_detection_color_set(rm_define.marker_detection_color_red)``

    :example description: Set the color for identifying visual labels to red

.. data:: color_enum

    +--------------------------------------+----+
    | rm_define.marker_detection_color_red | Red |
    +--------------------------------------+----+
    | rm_define.marker_detection_color_green | Green |
    +--------------------------------------+----+
    | rm_define.marker_detection_color_blue | Blue |
    +--------------------------------------+----+




- Example Code
Here's an example of how you might use this function to set the marker detection color to red:

In [ ]:
import robomaster
from robomaster import robot
from robomaster import vision
from robomaster import rm_define

def main():
    # Initialize the robot
    ep_robot = robot.Robot()
    ep_robot.initialize(conn_type="ap")

    # Initialize vision control
    vision_ctrl = ep_robot.vision

    # Set the marker detection color to red
    vision_ctrl.marker_detection_color_set(rm_define.marker_detection_color_red)

    # Example task: Detect and aim at a red marker
    vision_ctrl.detect_marker_and_aim(rm_define.marker_trans_red_heart)

    # Keep the program running
    try:
        while True:
            pass
    except KeyboardInterrupt:
        print("Exiting program")

    # Close the robot connection
    ep_robot.close()

if __name__ == '__main__':
    main()

In [ ]:
import cv2
import numpy as np
from robomaster import robot

def detect_black(frame):
    # Convert the frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the range for black color in HSV
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 30])

    # Create a mask for black color
    mask_black = cv2.inRange(hsv, lower_black, upper_black)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask_black, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Return the largest contour
    if contours:
        return max(contours, key=cv2.contourArea)
    return None

def main():
    # Initialize the robot
    ep_robot = robot.Robot()
    ep_robot.initialize(conn_type="ap")

    # Open the camera
    cap = cv2.VideoCapture(0)

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Detect black color
            black_contour = detect_black(frame)

            if black_contour is not None and cv2.contourArea(black_contour) > 500:
                # Draw the contour on the frame
                cv2.drawContours(frame, [black_contour], -1, (0, 0, 255), 3)

                # Example: Use the center of the contour to control the robot
                M = cv2.moments(black_contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    # Here you can add logic to move the robot based on cX, cY

            # Display the frame
            cv2.imshow('Frame', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except KeyboardInterrupt:
        print("Exiting program")

    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    ep_robot.close()

if __name__ == '__main__':
    main()